In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
np.random.seed(42)
np.random.RandomState(42)

RandomState(MT19937) at 0x7F039DF177C0

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
TRAIN_PATH = '/content/drive/MyDrive/potatos/train'
TEST_PATH = '/content/drive/MyDrive/Machine learning/Test.csv'
#SUB_PATH = '/content/drive/MyDrive/Machine learning/SampleSubmission.csv'

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(500, 500, 3),
    include_top=False)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    print(image.shape)
    plt.imshow(image)

In [ ]:
show_image('/content/drive/MyDrive/Colab Notebooks/potatos/train/Potato___Early_blight/Early_blight (12).jpg')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a data generator
datagen_train = ImageDataGenerator(
    samplewise_center=True,  # set each sample mean to 0
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,
)  # we don't expect Bo to be upside-down so we will not flip vertically

# No need to augment validation data
datagen_valid = ImageDataGenerator(samplewise_center=True)
datagen_test = ImageDataGenerator(samplewise_center=True)

In [ ]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/potatos/train/",
    target_size=(500 ,500),
    color_mode="rgb",
    class_mode="categorical",
#    batch_size=8,
)

# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/potatos/validation/",
    target_size=(500 ,500),
    color_mode="rgb",
    class_mode="categorical",
#    batch_size=8,
)

In [ ]:
test_it = datagen_test.flow_from_directory(
    "/content/drive/MyDrive/Colab Notebooks/potatos/test/test/",
    target_size=(500 ,500),
    color_mode="rgb",
    class_mode="categorical",
#    batch_size=8,
)

In [ ]:
base_model.trainable = False

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(500 ,500 , 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(3, activation = 'softmax')(x)

# Combine inputs and outputs to create model

model = keras.Model(inputs, outputs)

In [ ]:
from tensorflow.keras.losses import categorical_crossentropy

In [ ]:
model.compile(loss = categorical_crossentropy , metrics = 'accuracy')

In [ ]:
model.fit(train_it,
          validation_data= valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=20)

Epoch 1/20
20/20 [==============================] - 124s 6s/step - loss: 1.4025 - accuracy: 0.4955 - val_loss: 1.9749 - val_accuracy: 0.4808
Epoch 2/20
20/20 [==============================] - 107s 5s/step - loss: 1.1463 - accuracy: 0.5587 - val_loss: 1.6733 - val_accuracy: 0.5288
Epoch 3/20
20/20 [==============================] - 107s 5s/step - loss: 0.9830 - accuracy: 0.6099 - val_loss: 1.1791 - val_accuracy: 0.5577
Epoch 4/20
20/20 [==============================] - 107s 5s/step - loss: 0.8809 - accuracy: 0.6446 - val_loss: 1.0653 - val_accuracy: 0.5673
Epoch 5/20
20/20 [==============================] - 107s 5s/step - loss: 0.8461 - accuracy: 0.6657 - val_loss: 0.9548 - val_accuracy: 0.6250
Epoch 6/20
20/20 [==============================] - 107s 5s/step - loss: 0.7850 - accuracy: 0.6913 - val_loss: 1.2673 - val_accuracy: 0.6058
Epoch 7/20
20/20 [==============================] - 107s 5s/step - loss: 0.7058 - accuracy: 0.7093 - val_loss: 0.8362 - val_accuracy: 0.6827
Epoch 8/20
20

In [ ]:
#model.evaluate(test_it, steps= 24)  #test_it.samples/test_it.batch_size

In [ ]:
model.save('potatos')

INFO:tensorflow:Assets written to: potatos/assets


In [ ]:
model.evaluate(test_it, steps= test_it.samples/test_it.batch_size)

7/7 [==============================] - 7s 954ms/step - loss: 1.0650 - accuracy: 0.6429


[1.0650112628936768, 0.6428571343421936]